The purpose of this notebook is to generate the figures for the *Astyanax Mexicanus* social behavior paper using a pre-processed dataset.

Upstream of this notebook, we have tracked, processed, and analyzed data from the complete raw dataset. Here, we share data relevant to figures in a pickle file, `figure-data.pik`. Here, we show how to access and work with that file.

# Imports and module versions 

In [ ]:
import sys

# Print version numbers for the modules used in this notebook.
print(f'python {sys.version}')
for m in 'numpy', 'scipy', 'statsmodels', 'matplotlib', 'pandas', 'seaborn', 'pingouin':
    i = __import__(m)
    print(f'{m} {i.__version__}')

# Full imports.
import os
import sys
import pickle
import numpy as np
import scipy.interpolate
from scipy.stats import normaltest, shapiro, linregress
from scipy.stats import ttest_1samp, ttest_ind, ttest_ind_from_stats, f_oneway
from scipy.stats import mannwhitneyu
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests
from pingouin import ancova
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import hsv_to_rgb, rgb_to_hsv, to_rgb
import pandas as pd
import seaborn as sns
from collections import defaultdict, namedtuple
import itertools as itt
import copy


# T-test using means and std errors. The closest scipy equivalent requires
# means, std deviations, and number of observations.
TestResult = namedtuple('TestResult', 'statistic pvalue')
def ttest_from_err(mean1, stderr1, mean2, stderr2):
    t = (mean1-mean2)/np.sqrt(stderr1**2+stderr2**2)
    p = distributions.t.sf(np.abs(t), df) * 2

# ANOVA test using the samples' means, std deviations, and sizes.
# The closest scipy equivalent requires the original observations in each sample.
def f_oneway_from_stats(means, stds, counts):
    means  = np.asarray(list(means))
    stds   = np.asarray(list(stds))
    counts = np.asarray(list(counts), dtype=int)
    k      = len(means) # number of samples
    N      = np.sum(counts) # number of data points across all samples
    mean   = np.sum(counts*means)/np.sum(counts) # mean of all samples taken together
    ssb    = np.sum(counts*(means-mean)**2) # sum of squares between samples
    ssw    = np.sum((counts)*stds**2) # sum of squares within samples
    F      = (ssb/(k-1)) / (ssw/(N-k))
    pval   = 1 - scipy.stats.f.cdf(F, k-1, N-k)
    return TestResult(statistic=F, pvalue=pval)

# Mean and standard error.
def mean_sem(x, axis=None):
    return np.nanmean(x, axis=axis),scipy.stats.sem(x, axis=axis)

# Figure styling

Fonts, line styles, color schemes, etc.

In [ ]:
names       = { 'sf':'Surface', 'pa':'Pachón', 'mo':'Molino', 
                'ti':'Tinaja', 'sf-dark':'Surface in the dark' }
populations = list(names.keys())

plt.rc('font', size=16)
plt.rc('axes',  labelsize=16)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('legend', fontsize=16)
lw        = 2   # line width
dpi       = 150 # figure output resolution
alpha_err = 0.2 # transparency for std error halos
leg_opt   = dict( borderpad=0.3, labelspacing=0.2, handlelength=1.5,
                  handletextpad=0.5, borderaxespad=0.2, fontsize=14 ) # legend options
err_opt   = dict( lw=lw, elinewidth=lw, capthick=lw, capsize=5, barsabove=False ) # errorbar plot options
dashes    = dict(zip(populations, [(2,2), (3,1,1,1), (3,1,1,1,1,1), (1,1), (3,1)]))

def create_color_shades(rgb, gray=False):
    hsv = np.array([rgb_to_hsv(rgb)]*4)
    hsv[:,0] += np.array([-1,0,1,2])*0.02    # Slight hue gradient.
    hsv[:,1]  = np.array([0.5,0.6,0.9,1])    # Saturation gradient.
    hsv[:,2]  = np.array([0.9,0.75,0.5,0.2]) # Value gradient.
    if gray:
        hsv[:,1] = 0
        hsv[:,2]  = np.array([0.6,0.4,0.2,0]) # Value gradient.
    return np.array(list(map(hsv_to_rgb,hsv)))

def create_color_dictionaries(color_list, color_matrix, populations=populations):
    color_dict1  = dict(zip( populations, color_list )) # One shade per population.
    color_dict4  = { (pop,n):color_matrix[i,j] for i,pop in enumerate(populations) for j,n in enumerate([1,2,5,10]) }
    # Color map interpolating between white and the default shade.
    color_maps   = {}
    for k,c in color_dict1.items():
        # First max out saturation so colors "pop" a bit more, unless it's a grayscale family (s=0).
        c = rgb_to_hsv(c)
        if c[1]>0:
            c = hsv_to_rgb((c[0],1,c[2]))
        else:
            c = hsv_to_rgb((c[0],0,0.1))
        color_maps[k] = LinearSegmentedColormap.from_list(k,[c,(1,1,1)][::-1])
    return dict( color_dict1=color_dict1, color_dict4=color_dict4, color_maps=color_maps )

def visualize_color_scheme(color_list, color_matrix, title=None, ax=None):
    cl  = color_list.reshape((-1,1,3))
    mat = np.concatenate([color_matrix,np.ones_like(cl),cl],axis=1)
    if ax is None:
        ax = plt.gca()
    ax.imshow(mat)
    ax.set_xticks(range(6))
    ax.set_xticklabels(['1','2','3','4','','default'])
    ax.set_yticks([])
    ax.set_title(title, pad=10)

# Adam's color scheme.
color_matrix  = [ [ '#AADFFC', '#689BB7', '#315C72', '#09232F' ],  
                  [ '#FC7940', '#BF5428', '#823413', '#491702' ],
                  [ '#3AFAF2', '#4FB1A8', '#406E67', '#22332E' ],
                  [ '#F890A2', '#C75778', '#8C264E', '#4C0225' ],
                  [ '#59DA7A', '#FFFFFF', '#FFFFFF', '#0F3B1C' ] ]
color_matrix  = np.array([ [to_rgb(c) for c in C] for C in color_matrix ])
color_list    = color_matrix[:,1].copy()
color_list[4] = color_matrix[4,-1]
color_schemes = { 'adam': (color_list,color_matrix) }

# Scheme build on matplotlib's default line colors.
color_list = np.array(list(map(to_rgb, ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])))
color_matrix = np.array([create_color_shades(c) for c in color_list])
color_schemes['mpl_'] = (color_list,color_matrix)

# More matplotlib-based color schemes.
for k in ['tab20','tab20b','tab20c']:
    # Use the color scheme as-is.
    color_matrix = np.array(plt.get_cmap(k).colors).reshape((5,4,3))[:,::-1,:]
    color_list = color_matrix[:,-1,:].copy()
    color_schemes[k] = (color_list,color_matrix)
    # Keep each family's central hue, but apply custom recipe to create the shades within each family.
    color_list = np.array(plt.get_cmap(k).colors[::4])
    color_matrix = np.array([create_color_shades(c) for c in color_list])
    if k=='tab20c':
        color_matrix[4] = create_color_shades(color_list[4], gray=True)
    color_schemes[k+'_'] = (color_list,color_matrix)

# # Visualize every scheme under consideration.
# nc = 4
# nr = np.math.ceil(len(color_schemes)/nc)
# fig,axs = plt.subplots(nr, nc, figsize=(4*nc,4*nr), gridspec_kw=dict(wspace=0.5))
# for ax in axs.flatten():
#     ax.axis('off')
# for i,k in enumerate(color_schemes.keys()):
#     visualize_color_scheme(*color_schemes[k], title=k, ax=axs[i//nc,i%nc])

# Pick a scheme to use in the rest of the notebook.
globals().update(create_color_dictionaries(*color_schemes['mpl_']))

# Figure 1

In [ ]:
figure_data = pickle.load(open('data/figure1-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,7.5))
axs = axs.T

# Video snapshots.
for i in range(2):
    ax  = axs[0,i]
    pop = ['surface','pachon'][i]
    sf_track = mpimg.imread(f'assets/fig_snapshot-{pop}.png')
    ax.imshow(sf_track)
    ax.axis('off')

# Density and nematic OP.
for i in range(2):
    ax = axs[1,i]
    Q  = [density,nematic][i]
    for pop in populations[:-1]:
        mu,err = mean_sem(Q[pop], axis=0)
        ax.plot(bin_centers, mu, label=names[pop], 
                color=color_dict1[pop], lw=lw)
        ax.fill_between(bin_centers, mu-err, mu+err, 
                        color=color_dict1[pop], alpha=alpha_err, lw=0)
    ax.locator_params(axis='y', nbins=3)
    for y in [-1,0,1]:
        ax.axhline(y=y,color='k',dashes=(4,4),lw=0.5)
    ax.set_xlim(0,55)
    ax.set_xlabel('Distance from the wall (cm)')

ax = axs[1,0]
# ax.set_ylim(0,5.7e-4)
ax.set_ylim(0,9e-4)
ax.set_yticks([0,0.0004,0.0008])
ax.set_ylabel('Area density')

ax = axs[1,1]
ax.set_ylim(-1.1,0.55)
ax.set_yticks([-1,0])
ax.set_ylabel('Nematic order\nparameter')

# Legend.
plt.tight_layout()
axs[2,1].set_visible(False)
axs[1,1].legend(loc='center', bbox_to_anchor=(1.85,0.5),
                borderpad=1, framealpha=0.5, edgecolor='k', fancybox=True)

# Median distance to the wall.
ax   = axs[2,0]
med  = [ (k,x) for k in populations[:-1] for x in median[k]]
med  = pd.DataFrame(med, columns=['Population', 'Median distance\nfrom the wall (cm)'])
data = dict(x='Population', y='Median distance\nfrom the wall (cm)', data=med, ax=ax)
palette = [ v+0.2*(1-v) for v in color_dict1.values()]
sns.violinplot(inner=None, ec='None', palette=palette, cut=2, **data)
sns.stripplot(size=5, jitter=0.1, color='k', **data)
# ax.set_xlabel('')

y0 = 55.5/np.sqrt(2)
ax.axhline(y0, color='k', dashes=(4,4))
ax.text(3.5, y0+0.3, 'uniform area density ', fontsize=12, ha='right', va='bottom')

def draw_bracket(txt, x1, x2, y, dx=0, dy=0.8, ax=ax):
    # dy = ax.transAxes.inverted([0,dy_])
    ax.plot([x1+dx,x1+dx,x2-dx,x2-dx], [y-dy,y,y,y-dy], color='k', lw=0.5)
    ax.text((x1+x2)/2, y-1, txt, va='bottom', ha='center')
draw_bracket('***', 0, 1, 24)
draw_bracket('**', 0, 2, 28)
draw_bracket('*', 0, 3, 32)
ax.set_ylim(0, 46)

# Label each panel.
for i,c in enumerate('abcde'):
    ax = axs[i//2,i%2]
    ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

plt.savefig('fig1.png', dpi=dpi)
plt.show()

In [ ]:
# Test normality.
print('Shapiro test:')
for k,X in median.items():
    print(f'p[{k}] = {shapiro(X)[1]}')
# print(f'cave: p = {shapiro(df_cf)[1]}')

print('\nMann-Whitney test:')
for k in ['pa','mo','ti']:
#     p = ttest_ind(median['sf'],median[k]).pvalue
#     print(f'T-test sf-{k}: p = {p}')
    p = mannwhitneyu(median['sf'],median[k]).pvalue
    print(f'p[sf-{k}] = {p}')

# print('\nT-test:')
# for k in ['pa','mo','ti']:
#     p = ttest_ind(median['sf'],median[k]).pvalue
#     print(f'p[sf-{k}] = {p}')

# # If the hypothesis is that there's at least one inter-population difference, 
# # use a multiple test correction.
# # However here the hypothesis is that all three cave populations are different 
# # from the surface population. If anything the bar should be lowered.
# P = [ mannwhitneyu(median['sf'],median[k]).pvalue for k in ['pa','mo','ti'] ]
# multipletests(P) # , method='holm-sidak') # 'sidak') # 'bonferroni') # 

# Figure 2

While the cave fish tend to maintain an active swim state in our arena, regardless of group size, surface fish tend to stop swimming when they are alone or in small groups. In this figure we show the impact of this on the distribution of swim speed and consider the impact of inactivity across group size.

In [ ]:
figure_data = pickle.load(open('data/figure2-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

fig, axs = plt.subplots( nrows=1, ncols=2, figsize=(8,3.5), 
                         gridspec_kw=dict(wspace=0.5, bottom=0.2, top=0.95, right=0.95) )

ax = axs[0]
x  = speed_distribution['bin_centers']
for i,n in enumerate([1,2,5]): # Loop over group size.
    H      = speed_distribution['sf',n]
    mu,err = mean_sem(H, axis=0)
    ax.plot(x, mu, label=n, color=color_dict4['sf',n], lw=lw)
    ax.fill_between(x, mu-err, mu+err, color=color_dict4['sf',n], alpha=alpha_err, lw=0)
ax.set_xlabel('Speed (cm/s)')
ax.set_ylabel('Probability density')
ax.set_xlim(0,50)
# ax.set_ylim(0,0.25)
ax.set_ylim(0,None)
ax.locator_params('y',nbins=4)
ax.legend(**leg_opt)

ax     = axs[1]
n_list = [1,2,5,10]
F      = inactive_fraction
n_list = [1,2,5,10]
for pop in populations[:-1]:
    mu    = np.array([np.nanmean(F[pop,n], axis=0) for n in n_list])
    std   = np.array([np.nanstd(F[pop,n], axis=0) for n in n_list])
#     count = len(F[pop,n])
    count = np.count_nonzero(np.isfinite(F[pop,n]))
    err   = std / np.sqrt(count-1)
    ax.errorbar(n_list, mu, yerr=err, color=color_dict1[pop], label=names[pop], 
                ls=(0,dashes[pop]), **err_opt)
ax.set_ylim(None,1)
y1,y2 = ax.get_ylim()
ax.set_ylim(y1,y1+1.2*(y2-y1))
ax.set_xlabel('Group size')
ax.set_ylabel('Fraction of the\ntime active')
ax.set_xticks(n_list)
leg_opt.update(dict(handlelength=1.5))
ax.legend(**leg_opt)

# Label each panel.
for i,c in enumerate('ab'):
    ax = axs[i%2]
    ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

plt.tight_layout()
plt.savefig('fig2.png', dpi=dpi)
plt.show()

In [ ]:
figure_data = pickle.load(open('data/figure2-data.pik','rb'))
globals().update(figure_data)

ax = plt.gca()
x  = speed_distribution['bin_centers']
for i,n in enumerate([1,2,5]): # Loop over group size.
    H      = speed_distribution['sf',n]
    mu,err = mean_sem(H, axis=0)
    ax.plot(x, mu, label=n, color=color_dict4['sf',n], lw=lw)
#     ax.fill_between(x, mu-err, mu+err, color=color_dict4['sf',n], alpha=alpha_err, lw=0)
ax.set_xlabel('Speed (cm/s)')
ax.set_ylabel('Probability density')
ax.set_xlim(0,15)
ax.set_ylim(0,0.32)
ax.locator_params('y',nbins=4)
ax.legend(**leg_opt)
plt.show()

# Figure 3

Now let's take a look at the distribution of speeds after the activity cut. First we can grab histograms from the pickle.

In [ ]:
figure_data = pickle.load(open('data/figure3-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

n_list   = [1,2,5,10]
x        = speed_distribution['bin_centers']

fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
axs      = axs.flatten()

for i,pop in enumerate(['sf','pa']):
    ax  = axs[i]
    ax.set_title(names[pop])
    ax.set_xlabel('Speed (cm/s)')
    ax.set_ylabel('Probability density')
    for n in n_list:
        H     = speed_distribution[pop,n]
        y,err = mean_sem(H, axis=0)
        ax.plot(x, y, label=n, color=color_dict4[pop,n], lw=lw)
        ax.fill_between(x, y-err, y+err, color=color_dict4[pop,n], alpha=alpha_err, lw=0, zorder=-5)
    ax.legend(**leg_opt)
    ax.set_xlim(0,75)
    ax.set_ylim(0,1.1*ax.get_ylim()[1])
    ax.locator_params('y',nbins=4)

ax = axs[2]
ms_stats = {}
for pop in populations[:-1]:
#     xy = [(n,m) for n in n_list for m in means[pop,n,'speed'][:,0]]
#     ax.scatter(*zip(*xy), color=color_dict1[pop], s=3)
#     MS  = [means[pop,n,'speed'][:,0] for n in n_list]
    MS  = [mean_speed[pop,n] for n in n_list]
    y   = [np.mean(ms) for ms in MS]
    err = [np.std(ms)/np.sqrt(len(ms)-1) for ms in MS]
    ax.errorbar(n_list, y, yerr=err, color=color_dict1[pop], label=names[pop], 
                alpha=0.7, ls=(0,dashes[pop]), **err_opt)
ax.set_xlabel('Group size')
ax.set_ylabel('Mean speed when\nactive (cm/s)')
ax.set_xticks(n_list)
ax.set_ylim(0,28)
ax.locator_params(axis='y', nbins=4)
ax.legend(**leg_opt)

# Label each panel.
for i,c in enumerate('abc'):
    ax = axs[i] # [i//2,i%2] 
    ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

plt.tight_layout()
plt.savefig('fig3.png', dpi=dpi)
plt.show()

In [ ]:
slope   = []
# fig,axs = plt.subplots(2, 2, figsize=(6,6))
fig,axs = plt.subplots(1, 4, figsize=(12,3))
for ax,pop in zip(axs.flatten(),populations[:-1]):
#     x,y  = np.array([ (k[1],m) for k,M in means.items() if k[0]==pop and k[2]=='speed' for m in M[:,0] ]).T
    x,y  = np.array([ (n,m) for (pop_,n),M in mean_speed.items() if pop_==pop for m in M ]).T
#     x,y  = np.array([ (n,m) for n in [1,2,5,10] for m in means[pop,n,'speed'][:,0] ]).T
    ax.scatter(x, y, fc='None', ec=color_dict1[pop], lw=1, label=names[pop])
    reg = linregress(x, y) #, alternative='less')
    slope.append([pop, reg.slope, reg.stderr, len(x), reg.pvalue])
    ax.plot(x, reg.slope*x+reg.intercept, color=color_dict1[pop])
    ax.set_title(names[pop])
    ax.set_xlabel('Group size')
    ax.set_xticks([1,2,5,10])
    if pop=='sf':
        ax.set_ylabel('Mean speed when\nactive (cm/s)')
    ax.set_ylim(0,38)
# fig.legend(loc='center left', bbox_to_anchor=(0.95,0.5))
plt.tight_layout()
plt.show()

print(f'p-values for a nonzero slope:')
for pop,s,err,_,pval in slope:
    print(f'  {pop}: slope={s:<.1f}+-{err:.1f}, p-value={pval:<.3g}')

ax = plt.gca()
n,s,e,no,p = zip(*slope)
ax.errorbar(n, s, yerr=e, **err_opt)
ax.set_xticklabels(names.values())
ax.set_xlabel('Population')
ax.set_ylabel('Mean speed slope\n(cm/s/fish)')
plt.show()

# Figure S3

In [ ]:
figure_data = pickle.load(open('data/figure3-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

n_list   = [1,2,5,10]
x        = speed_distribution['bin_centers']

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,7))

for i,pop in enumerate(populations[:-1]):
    ax = axs[i//2,i%2]
    ax.set_title(names[pop])
    ax.set_xlabel('Speed (cm/s)')
    ax.set_ylabel('Probability density')
    print(pop, speed_distribution[pop,10].shape)
    for n in n_list:
        H     = speed_distribution[pop,n]
        y,err = mean_sem(H, axis=0)
        ax.plot(x, y, label=n, color=color_dict4[pop,n], lw=lw)
        ax.fill_between(x, y-err, y+err, color=color_dict4[pop,n], alpha=alpha_err, lw=0, zorder=-5)
    ax.legend(**leg_opt)
    ax.set_xlim(0,75)
    ax.set_ylim(0,None)
    ax.locator_params('y',nbins=4)
    ax.set_ylim(0,1.1*ax.get_ylim()[1])

# Label each panel.
for i,c in enumerate('abcd'):
    ax = axs[i//2,i%2] 
    ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

plt.tight_layout()
plt.savefig('figS3.png', dpi=dpi)
plt.show()

In [ ]:
# ''' Look at distributions from individual trials. '''

# figure_data = pickle.load(open('data/figure3-data.pik','rb'))
# # print(figure_data.keys())
# globals().update(figure_data)

# n_list   = [10]
# x        = speed_distribution['bin_centers']

# fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,7))

# for i,pop in enumerate(populations[:-1]):
#     ax = axs[i//2,i%2]
#     ax.set_title(names[pop])
#     ax.set_xlabel('Speed (cm/s)')
#     ax.set_ylabel('Probability density')
#     print(pop, speed_distribution[pop,10].shape)
#     for n in n_list:
#         for y in speed_distribution[pop,n]:
#             ax.plot(x, y, color=color_dict4[pop,10], lw=lw)
# #     ax.legend(**leg_opt)
#     ax.set_xlim(0,75)
#     ax.set_ylim(0,None)
#     ax.locator_params('y',nbins=4)
#     ax.set_ylim(0,1.1*ax.get_ylim()[1])

# # Label each panel.
# for i,c in enumerate('abcd'):
#     ax = axs[i//2,i%2] 
#     ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

# plt.tight_layout()
# plt.savefig('figS3.png', dpi=dpi)
# plt.show()

# Figure 4

In this figure, we focus on the distributions of turning behavior across type and group size.

In [ ]:
figure_data = pickle.load(open('data/figure4-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

n_list   = [1,2,5,10]
x        = angSpeed_distribution['bin_centers']

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,8),
                        gridspec_kw={'width_ratios': [8,20], 'wspace':0.6, 
                                     'hspace':0.5, 'right':0.95, 'top':0.95})
for i,pop in enumerate(['sf','pa']):
    ylabel = 'Time-based\nprobability density', 'Angle-based\nprobability density'
    xlim   = (0,8), (0,20)
    for j,ax in enumerate(axs[i,:]):
        ax = axs[i,j]
        ax.set_title(names[pop])
        ax.set_xlabel('Angular speed (rad/s)')
        ax.set_xlim(xlim[j])
        ax.set_ylabel(ylabel[j])
        ax.locator_params(axis='y', nbins=5)
    for n in n_list:
        H     = angSpeed_distribution[pop,n]
        y,err = mean_sem(H, axis=0)
        axs[i,0].plot(x, y, label=n, color=color_dict4[pop,n], lw=lw)
        axs[i,0].fill_between(x, y-err, y+err, color=color_dict4[pop,n], 
                              alpha=alpha_err, lw=0, zorder=-5)
        a     = 1/np.trapz(x*y, x) # Density normalization.
        axs[i,1].plot(x, a*x*y, label=n, color=color_dict4[pop,n], lw=lw)
        axs[i,1].fill_between(x, a*x*(y-err), a*x*(y+err), color=color_dict4[pop,n], 
                              alpha=alpha_err, lw=0, zorder=-5)
    axs[i,1].legend(loc='upper right', **leg_opt)

# Vertical dashes lines.
for ax in axs.flatten():
    for x0 in [1,7]:
        ax.plot([x0,x0],ax.get_ylim(), lw=0.7*lw, color='k', dashes=(4,4))
    ax.set_xlabel('Angular speed (rad/s)')
    ax.set_ylim(0,1.1*ax.get_ylim()[1])
    ax.locator_params(axis='x', nbins=5)

# Label each panel.
for i,c in enumerate('abcd'):
    ax = axs[i//2,i%2] 
    t = ax.text(0.05/(i%2+1), 0.98, c, transform=ax.transAxes, 
                fontsize=24, weight='bold', va='top')

plt.tight_layout()
plt.savefig('fig4.png', dpi=dpi)
plt.show()

# Figure S4

In [ ]:
figure_data = pickle.load(open('data/figure4-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

n_list   = [1,2,5,10]
x        = angSpeed_distribution['bin_centers']

fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(8,16),
                       gridspec_kw={'width_ratios': [8,20], 'wspace':0.6, 'hspace':0.5,
                                    'right':0.95, 'top':0.95})
for i,pop in enumerate(populations[:-1]):
    ax = axs[i,0]
    ax.set_title(names[pop])
    ax.set_xlabel('Angular speed (rad/s)')
    ax.set_ylabel('Time-based\nprobability density')
    for n in n_list:
        H     = angSpeed_distribution[pop,n]
        y,err = mean_sem(H, axis=0)
        ax.plot(x, y, label=n, color=color_dict4[pop,n], lw=lw)
        ax.fill_between(x, y-err, y+err, color=color_dict4[pop,n], alpha=alpha_err, lw=0, zorder=-5)
    ax.set_xlim(0,8)
    
    ax = axs[i,1]
    ax.set_title(names[pop])
    ax.set_ylabel('Angle-based\nprobability density')
    for n in n_list:
        H     = angSpeed_distribution[pop,n]
        y,err = mean_sem(H, axis=0)
        a   = 1/np.trapz(x*y, x) # Density normalization.
        ax.plot(x, a*x*y, label=n, color=color_dict4[pop,n], lw=lw)
        ax.fill_between(x, a*x*(y-err), a*x*(y+err), color=color_dict4[pop,n], alpha=alpha_err, lw=0, zorder=-5)
    ax.set_xlim(0,20)
    ax.legend(loc='upper right', **leg_opt)
    
# Vertical dashes lines.
for ax in axs.flatten():
    for x0 in [1,7]:
        ax.plot([x0,x0],ax.get_ylim(), lw=0.7*lw, color='k', dashes=(4,4))
    ax.set_xlabel('Angular speed (rad/s)')
    ax.set_ylim(0,1.1*ax.get_ylim()[1])
    ax.locator_params(axis='x', nbins=5)

# Label each panel.
for i,c in enumerate('abcdefgh'):
    ax = axs[i//2,i%2] 
    ax.text(0.05/(i%2+1), 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

plt.tight_layout()
plt.savefig('figS4.png', dpi=dpi)
plt.show()

# Figure 5

We're interested in the collective behavior. To summarize the collective schooling and/or shoaling behavior of these fish, we measure relative distance and relative angle between all pairs of fish.

In [ ]:
figure_data = pickle.load(open('data/figure5-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

n_list   = [1,2,5,10]
x, y     = bins_d, bins_a

gridspec=dict(wspace=0.4, hspace=0.4, left=0.02, right=0.98, bottom=0.08, top=0.94)
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,8), gridspec_kw=gridspec)
fig.tight_layout()

# Label each panel.
# Note: with `transform=ax.transAxes`, the labels move if the axes object is later resized.
# Doing the coordinate transform explicitly (after calling `tight_layout`) avoids that,  
# thus keeping the labels on a regular grid.
for i,c in enumerate('abcdef'):
    ax = axs[i//3,i%3] 
    xy = fig.transFigure.inverted().transform(ax.transAxes.transform([0.02,0.98]))
    ax.text(*xy, c, transform=fig.transFigure, fontsize=24, weight='bold', va='top')

# First column: schematics.
for i,name in enumerate(['pair-angle','collective-behavior']):
    axs[i,0].imshow(mpimg.imread(f'assets/fig_schematic_{name}.png'))
    axs[i,0].axis('off')
bbox = axs[1,0].get_position()
bbox.y0,bbox.y1 = np.array([bbox.y0,bbox.y1]) - 0.04
axs[1,0].set_position(bbox)

for i,pop in enumerate(['sf','pa']):
    for j,n in enumerate([2,10]):
        ax   = axs[j,i+1]
        z    = distAng_heatmap[pop,n].T
        '''
        !!!!!!
        The percentile trick works around a small number of very large values.
        It was not needed in the original data. It's most likely due to the
        lack of overlap cut.
        !!!!!!
        '''
        vmax = np.percentile(z,99.9) # np.max(z) # 
        ax.pcolormesh(x, y, z, cmap = color_maps[pop], vmin=0, vmax=vmax)
        ax.set_xlim(x[0], x[-1])
        ax.set_ylim(y[0], y[-1])
        ax.set_yticks([0,30,60,90,120,150,180])
        ax.set_ylabel("Pair angle (deg)")
        ax.set_xlabel("Pair distance (cm)")
        ax.set_title(f'{n} {names[pop]}')

# Wall following prediction (arches).
D     = 111  # tank diameter
D_    = 103 # effective tank diameter (wall-following diameter)
d     = np.arange(0,D+1,1)
theta = (180/np.pi) * np.arccos(1 - 2*(d/D_)**2)
opt   = dict(lw=1.5, alpha=0.7, color='k', dashes=(4,4))
for i in range(2):
    axs[i,2].plot(d, theta, **opt)
    axs[i,2].plot(d, 180-theta, **opt)

plt.savefig('fig5.png', dpi=dpi)
plt.show()

# Figure S5

In [ ]:
figure_data = pickle.load(open('data/figure5-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

n_list   = [1,2,5,10]
x, y     = bins_d, bins_a

for n in [2,5,10]:
    gridspec = dict(wspace=0.4, hspace=0.4, left=0.1, right=0.97, bottom=0.08, top=0.94)
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,8), gridspec_kw=gridspec)
    fig.tight_layout()

    # Label each panel.
    for i,c in enumerate('abcd'):
        ax = axs[i//2,i%2] 
        ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

    for i,pop in enumerate(populations[:-1]):
        ax  = axs[i//2,i%2]
        z    = distAng_heatmap[pop,n].T
        vmax = np.percentile(z,99.9) # np.max(z) # 
        ax.pcolormesh(x, y, z, cmap = color_maps[pop], vmin=0, vmax=vmax)
        ax.set_xlim(x[0], x[-1])
        ax.set_ylim(y[0], y[-1])
        ax.set_yticks([0,30,60,90,120,150,180])
        ax.set_ylabel("Pair angle (deg)")
        ax.set_xlabel("Pair distance (cm)")
        ax.set_title(f'{n} {names[pop]}')

    plt.savefig(f'figS5-n{n:02}.png', dpi=dpi)
    plt.show()

# Figure 6

In this figure, we consider surface fish in a dark tank and compare some aspects of their behavior to cavefish.

In [ ]:
bins_d

In [ ]:
# Load data.
figure_data = pickle.load(open('data/figure3-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)
figure_data = pickle.load(open('data/figure5-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

# Create figure.
gridspec = dict(wspace=0.4, hspace=0.4, left=0.07, right=0.99, bottom=0.08, top=0.9)
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12,3.8), gridspec_kw=gridspec)
fig.tight_layout()
inset_fontsize = 12
# err_opt2 = dict(lw=lw, elinewidth=0.7*lw, capsize=2) # inset error bar options
err_opt2 = dict(lw=lw, elinewidth=0.7*lw, capsize=3) # inset error bar options

# Label each panel.
for i,c in enumerate('abc'):
    ax = axs[i] 
    ax.text(0.02, 0.98, c, transform=ax.transAxes, 
            fontsize=24, weight='bold', va='top')

# Left panel: speed distribution.
ax  = axs[0]
D   = 111 # tank diameter
pop = 'sf-dark'
x   = speed_distribution['bin_centers']
for n in [1,10]:
    H     = speed_distribution[pop,n]
    y,err = mean_sem(H, axis=0)
    ax.plot(x, y, label=n, color=color_dict4[pop,n], lw=lw)
    ax.fill_between(x, y-err, y+err, color=color_dict4[pop,n], 
                    alpha=alpha_err, lw=0, zorder=-5)
ax.set_xlim(0,65)
ax.set_ylim(0,1.1*ax.get_ylim()[1])
ax.locator_params('y',nbins=5)
ax.set_xlabel('Speed (cm/s)')
ax.set_ylabel('Probability density')
# leg_opt   = dict( borderpad=0.3, labelspacing=0.2, handlelength=1.5,
#                   handletextpad=0.5, borderaxespad=0.2, fontsize=15 )
l_opt = dict( borderpad=0.3, labelspacing=0.2, handlelength=1,
              handletextpad=0.5, borderaxespad=0.2, fontsize=15 )
ax.legend(loc='lower left', **l_opt)
ax.set_title(names[pop])

# Left panel inset: mean speed vs group size.
x0,y0,x1,y1 = axs[0].get_position()._points.flatten()
x0_,y0_     = x0+0.45*(x1-x0), y0+0.33*(y1-y0)
ax_         = fig.add_axes([x0_, y0_, x1-x0_-0.003, y1-y0_-0.01])
# Prepare data.
process_means = lambda n,m: (n, np.mean(m), np.std(m)/np.sqrt(len(m)-1))
  # Surface in the light.
x,y,err = np.array([ process_means(n, mean_speed['sf',n]) for n in [1,2,5,10] ]).T
ax_.errorbar(x, y, yerr=err, color=color_dict1['sf'], 
             ls=(0,dashes['sf']), label='Surface, light', **err_opt2)
  # Surface in the dark.
x,y,err = np.array([ process_means(n, mean_speed['sf-dark',n]) for n in [1,10] ]).T
ax_.errorbar(x, y, yerr=err, color=color_dict1['sf-dark'], 
             ls=(0,dashes['sf-dark']), label='Surface, dark', **err_opt2)
  # All cavefish.
# x,y,err = np.array([ process_means(n, trial_store['means-vcut']['all-cavefish',n][:,0]) for n in [1,10] ]).T
means   = lambda n: [ m for pop in ['pa','mo','ti'] for m in mean_speed[pop,n] ]
x,y,err = np.array([ process_means(n,means(n)) for n in [1,2,5,10] ]).T
ax_.errorbar(x, y, yerr=err, color='#AA0000', 
             ls=(0,dashes['pa']), label='All cavefish', **err_opt2)
ax_.set_xlim(0,11)
ax_.set_ylim(0,None)
ax_.set_xticks([1,2,5,10])
ax_.set_xlabel('Group size', fontsize=inset_fontsize, labelpad=0)
ax_.set_ylabel('Mean speed (cm/s)', fontsize=inset_fontsize, labelpad=0, y=0.6)
l_opt = dict( borderpad=0.3, labelspacing=0.2, handlelength=1.5,
              handletextpad=0.5, borderaxespad=0.2, fontsize=inset_fontsize, 
              frameon=False )
ax_.legend(**l_opt)

# Middle panel: distance-angle probability heatmap.
ax    = axs[1]
pop,n = 'sf-dark',10
x, y  = bins_d, bins_a
z     = distAng_heatmap[pop,n].T
vmax  = np.percentile(z,99.9) # np.max(z) # 
ax.pcolormesh(x, y, z, cmap = color_maps[pop], vmin=0, vmax=vmax)
ax.set_xlim(x[0], x[-1])
ax.set_ylim(y[0], y[-1])
ax.set_yticks([0,30,60,90,120,150,180])
ax.set_ylabel("Pair angle (deg)")
ax.set_xlabel("Pair distance (cm)")
ax.set_title(f'{n} {names[pop]}')

# Right panel: angle probability when close.
ax   = axs[2]
n    = 10 # group size
x    = (bins_a[:-1]+bins_a[1:])/2
for pop in populations:
    y,err = mean_sem(ang_distribution[pop,n], axis=0)
    ax.plot(x, y, label=pop, color=color_dict1[pop], lw=lw)
    ax.fill_between(x, y-err, y+err, color=color_dict1[pop], 
                    alpha=alpha_err, lw=0, zorder=-5)
ax.set_xlim(0,180)
ax.set_ylim(0,1.1*ax.get_ylim()[1])
ax.locator_params('y',nbins=5)
ax.set_xlabel(r'Pair angle $\theta$ (deg)')
ax.set_ylabel('Probability density')
ax.set_title(f'{n} fish')


# Right panel inset: mean cosine vs group size.
x0,y0,x1,y1 = axs[2].get_position()._points.flatten()
x0_,y0_ = x0+0.35*(x1-x0), y0+0.5*(y1-y0)
ax_     = fig.add_axes([x0_, y0_, x1-x0_-0.003, y1-y0_-0.01])

# x,c,y,e = zip(*[ (names[pop],color_dict1[pop])+mean_sem(v) 
#                  for (pop,n),v in polar.items() if n==10 ])

P       = defaultdict(list)
for (pop,n),v in polar.items():
    P[pop] += v.tolist()
x,c,y,e = zip(*[ (names[pop],color_dict1[pop])+mean_sem(v) 
                 for pop,v in P.items() ])

x = x[:-1] + ('Surface (dark)',)
ax_.bar(x, y, color=c)
ax_.errorbar(x, y, yerr=e, fmt='none', color='k', **err_opt2)
ax_.axhline(0, color='k', lw=0.5)
ax_.set_xticklabels(x, rotation=45, ha='right', y=0.05)
ax_.set_yticks([0,1])
ax_.set_ylim(-0.1,1.25)
ax_.set_ylabel(r'$\langle\cos\,\theta\rangle$', labelpad=0)

def draw_bracket(txt, x1, x2, y, dx=0, dy=0.05, dyt=0, ax=ax_, **args):
    ax.plot([x1+dx,x1+dx,x2-dx,x2-dx], [y-dy,y,y,y-dy], color='k', lw=0.5)
    ax.text((x1+x2)/2, y+dyt, txt, va='bottom', ha='center', **args)
draw_bracket('n.s.', 1, 3, 0.35, dx=-0.2, fontsize=12, dyt=0.01)
# draw_bracket('', 1, 3, 0.4, dx=-0.2)
# ax_.text(2, 0.38, 'Cave', va='top', ha='center', fontsize=12)
draw_bracket('***', 0, 1.8, 1, dyt=-0.05)
ax_.plot([1.8]*2, [1,0.55], color='k', lw=0.5)
draw_bracket('***', 2.2, 4, 0.6, dyt=-0.05)
ax_.plot([2.2]*2, [0.6,0.55], color='k', lw=0.5)


plt.savefig('fig6.png', dpi=dpi)
plt.show()

In [ ]:
# Statistical tests for <v(n)> (left panel, inset).

''' !!! To do !!! '''

In [ ]:
# Statistical tests for <cos(theta)> (right panel, inset).

figure_data = pickle.load(open('data/figure5-data.pik','rb'))
# print(figure_data.keys())
globals().update(figure_data)

# plt.boxplot(polar.values(), labels=polar.keys())
# plt.xticks(rotation=45, ha='right')
# plt.show()

polar = { k:v for k,v in polar.items() if k[1]==10 }

# Test normality.
print('Shapiro test:')
for k,V in polar.items():
    print(f'{k[0]}: p = {shapiro(V)[1]:.2f}')

# Make DataFrame for Tukey test.
df = pd.DataFrame([k+(v,) for k,V in polar.items() for v in V], 
                  columns=['pop','n_ind','cos_mean'])
print()
print(pairwise_tukeyhsd(df['cos_mean'], df['pop']))

cave = ['pa','mo','ti']
p = f_oneway(*[V for (pop,n),V in polar.items() if pop in cave]).pvalue
print(f'\nOne-way ANOVA test among cavefish populations: p = {p}')

# Merge cave populations and perform Tukey test.
df['pop'] = df['pop'].apply(lambda k: 'cave' if k in cave else k)
print()
print(pairwise_tukeyhsd(df['cos_mean'], df['pop']))

# Uncorrected pairwise T-tests.
gb = df.groupby('pop')['cos_mean']
for p1,p2 in itt.combinations(gb.groups.keys(),2):
    p = ttest_ind(gb.get_group(p1),gb.get_group(p2)).pvalue
    print(f'{p1}-{p2}: {p:.2g}')


# [Work in progress]

Figures above use trilab-tracker data.
Figures below use Adam's figure data.

# Figure 7

Here, we consider the effect of evasive interactions on time spent in the center of the tank. Experiments show that cave fish spend more time in the center of the tank with increased group size. We compare time in the center across simulations of three variants of a minimal active matter model. 

Where we consider time in the center of the tank of radius $R$, where the center is defined so that there is an equal area in the center compared to outside of the tank.
\begin{align}
A_\text{center} &= A_\text{total} /2 \\
R_\text{center}^2 &= R^2 /2 \\
R_\text{center} &= R / \sqrt{2} 
\end{align}

If fish were to spend time uniformly throughout the tank, then the probability of finding the fish in the center and near the walls would be equal.

In [ ]:
gridspec = dict(wspace=0.4, bottom=0.15, left=0.01, right=0.99, top=0.97)
fig, axs = plt.subplots(ncols=3, nrows=1, figsize=(12,3.8), gridspec_kw=gridspec)
plt.tight_layout()

# Label each panel.
for ax,c in zip(axs,'abc'):
    ax.text(0.02, 0.98, c, transform=ax.transAxes, fontsize=24, weight='bold', va='top')

    
# Left panel: Overlaid screenshots of evasive maneuver.
ax = axs[0]
evasive_tracks = mpimg.imread(fpath('fig_evasion.png'))
ax.imshow(evasive_tracks)
ax.axis('off')


# Middle panel: Time in center vs group size (experiments).
ax  = axs[1]
d_expt = trial_store['time-in-center_expt']
n_list = [1,2,5,10]
for pop in ['Pa','Mo','Ti']:
    y   = [ np.mean(d_expt[pop,n]) for n in n_list ]
    err = [ np.std(d_expt[pop,n])/np.sqrt(len(d_expt[pop,n])-1) for n in n_list ]
    ax.errorbar(n_list, y, yerr=err, label=names[pop], dashes=dashes[pop], 
                color=color_dict1[pop], alpha=0.8, **err_opt)
ax.legend(loc='upper center', **leg_opt)


# Right panel: Time in center vs group size (simulations).
ax          = axs[2]
fic         = trial_store['time-in-center_sims']
fic_models  = ['ignore', 'slow', 'avoid', 'combo']
fic_markers = ['o', 's', '^', 'p']
fic_colors  = ['grey', 'r', 'b', 'magenta']
fic_names   = ['Ignore', 'Slowdown', 'Evade', 'Combined']
for i,m in enumerate(fic_models):
    x,y,err = zip(*[ (k[1], np.mean(f), np.std(f)/np.sqrt(len(f)-1)) for k,f in fic.items() if k[0]==m])
#     ax.errorbar(x, y, yerr=err, label=fic_names[i], color=fic_colors[i], marker=fic_markers[i], ms=7, **err_opt)
    ax.plot(x, y, label=fic_names[i], color=fic_colors[i], alpha=0.8, marker=fic_markers[i], ms=7)
ax.legend(loc='upper left', bbox_to_anchor=(0.1, 1), **leg_opt)


# Common settings for middle and right panels.
for ax in axs[1:]:
    ax.set_xlabel('Group size')
    ax.set_ylabel('Fraction of time in center')
    ax.set_xlim(0,12.5)
    ax.set_ylim(0,0.55)
    ax.set_xticks([1,2,5,10])
    ax.set_yticks([0,0.2,0.4])
#     ax.legend(loc='upper left', bbox_to_anchor=(0.17,1), **leg_opt)


plt.savefig(fpath("fig7.png"), dpi=dpi)
plt.show()